In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.types import StructType, StructField, BooleanType, StringType, IntegerType, DateType, FloatType,DoubleType,ArrayType,LongType
import logging
import sys
import traceback
import ast
import json
import os
import sys
import traceback
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr,when,to_date ,udf, concat_ws,posexplode, from_json
from pyspark.sql.types import StructType, StructField, BooleanType, StringType, IntegerType, DateType, FloatType,DoubleType
from pyspark.sql import functions as F

from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, DoubleType
from pyspark.ml.linalg import Vectors

In [2]:
# spark = SparkSession.builder \
#     .appName("MinIO with Delta Lake BERT") \
#     .config("spark.driver.memory", "8g") \
#     .config("spark.executor.memory", "8g") \
#     .config("spark.driver.cores", "6") \
#     .config("spark.executor.cores", "6") \
#     .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3") \
#     .config("spark.dynamicAllocation.enabled", "true") \
#     .config("spark.dynamicAllocation.minExecutors", "1") \
#     .config("spark.dynamicAllocation.maxExecutors", "1") \
#     .config("spark.dynamicAllocation.initialExecutors", "1") \
#     .config("spark.sql.shuffle.partitions", "12") \
#     .config("spark.memory.offHeap.enabled", "true") \
#     .config("spark.memory.offHeap.size", "4g") \
#     .config("spark.jars", "jars/hadoop-aws-3.3.4.jar,jars/spark-sql-kafka-0-10_2.12-3.2.1.jar,jars/aws-java-sdk-bundle-1.12.262.jar,jars/delta-core_2.12-2.2.0.jar,jars/delta-storage-2.2.0.jar") \
#     .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
#     .config("spark.hadoop.fs.s3a.access.key", "conbo123") \
#     .config("spark.hadoop.fs.s3a.secret.key", "123conbo") \
#     .config("spark.hadoop.fs.s3a.path.style.access", "true") \
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#     .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
#     .config("delta.enable-non-concurrent-writes", "true") \
#     .config('spark.sql.warehouse.dir', "s3a://lakehouse/") \
#     .config("spark.sql.shuffle.partitions", "100") \
#     .getOrCreate()


spark = SparkSession.builder \
    .appName("HybridRecommenderMaxPerformance") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "6g") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.instances", "1") \
    .config("spark.default.parallelism", "8") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "4g") \
    .config("spark.jars", "jars/hadoop-aws-3.3.4.jar,jars/spark-sql-kafka-0-10_2.12-3.2.1.jar,jars/aws-java-sdk-bundle-1.12.262.jar,jars/delta-core_2.12-2.2.0.jar,jars/delta-storage-2.2.0.jar") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "conbo123") \
    .config("spark.hadoop.fs.s3a.secret.key", "123conbo") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    .config("delta.enable-non-concurrent-writes", "true") \
    .config("spark.sql.warehouse.dir", "s3a://lakehouse/") \
    .getOrCreate()

25/05/25 06:28:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, StopWordsRemover
from pyspark.ml import Pipeline
spark_df = spark.read.format("delta").load("s3a://lakehouse/gold/machineData")

tokenizer = Tokenizer(inputCol="comb", outputCol="words")
stopword_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
hashingTF = HashingTF(inputCol="filtered_words", outputCol="rawFeatures", numFeatures=1000)
idf = IDF(inputCol="rawFeatures", outputCol="features")
pipeline = Pipeline(stages=[
    tokenizer, 
    stopword_remover, 
    hashingTF, 
    idf])
pipeline_mdl = pipeline.fit(spark_df)
new_df = pipeline_mdl.transform(spark_df)

In [4]:
# Định nghĩa UDF để chuyển sparse vector thành dense vector
def to_dense(sparse_vector):
    return Vectors.dense(sparse_vector).tolist()

to_dense_udf = udf(to_dense, ArrayType(DoubleType()))

# Áp dụng UDF để tạo cột 'vecs'
new_df_with_dense = new_df.withColumn("vecs", to_dense_udf("features"))

In [5]:
new_df_with_dense.select('id', 'vecs').write.format("delta").mode("overwrite").save("s3a://lakehouse/data/all_movies_delta_IDF")

In [ ]:
from pyspark.ml.feature import Tokenizer, CountVectorizer, IDF, StopWordsRemover, PCA
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors
from pyspark.sql.types import ArrayType, DoubleType

# === B1: Đọc dữ liệu ===
spark_df = spark.read.format("delta").load("s3a://lakehouse/gold/machineData")

tokenizer = Tokenizer(inputCol="comb", outputCol="words")
stopword_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
count_vectorizer = CountVectorizer(inputCol="filtered_words", outputCol="rawFeatures", vocabSize=5000)
idf = IDF(inputCol="rawFeatures", outputCol="features")

pipeline = Pipeline(stages=[
    tokenizer,
    stopword_remover,
    count_vectorizer,
    idf
])

pipeline_model = pipeline.fit(spark_df)
transformed_df = pipeline_model.transform(spark_df)

def to_list(vector):
    return vector.toArray().tolist()

to_list_udf = F.udf(to_list, ArrayType(DoubleType()))
new_df_with_dense = transformed_df.withColumn("vecs", to_list_udf("features")).select("id", "vecs")
new_df_with_dense.write.format("delta").mode("overwrite").save("s3a://lakehouse/data/tfidf")


# pca = PCA(k=500, inputCol="features", outputCol="pca_features")  # Output tạm

# ,
#     pca



25/05/25 06:28:40 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/05/25 06:28:47 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
pipeline_model = pipeline.fit(spark_df)
transformed_df = pipeline_model.transform(spark_df)
def to_list(vector):
    return vector.toArray().tolist()

to_list_udf = F.udf(to_list, ArrayType(DoubleType()))
new_df_with_dense = transformed_df.withColumn("vecs", to_list_udf("features")).select("id", "vecs")
new_df_with_dense.write.format("delta").mode("overwrite").save("s3a://lakehouse/data/tfidf")

25/05/25 06:34:35 WARN HeartbeatReceiver: Removing executor 1 with no recent heartbeats: 302924 ms exceeds timeout 120000 ms
25/05/25 06:34:35 WARN HeartbeatReceiver: Removing executor 0 with no recent heartbeats: 302821 ms exceeds timeout 120000 ms
25/05/25 06:34:35 ERROR Inbox: Ignoring error
java.lang.AssertionError: assertion failed: BlockManager re-registration shouldn't succeed when the executor is lost
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$register(BlockManagerMasterEndpoint.scala:632)
	at org.apache.spark.storage.BlockManagerMasterEndpoint$$anonfun$receiveAndReply$1.applyOrElse(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:103)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:100)
	at org.apache.spark.rpc.netty.MessageLoop.org$apache$s

In [5]:
new_df_with_dense.write.format("delta").mode("overwrite").save("s3a://lakehouse/data/tfidf")

25/05/25 07:33:50 WARN DAGScheduler: Broadcasting large task binary with size 19.6 MiB


In [6]:
t= spark.read.format("delta").load("s3a://lakehouse/data/tfidf")
t.show()

+---+--------------------+
| id|                vecs|
+---+--------------------+
| 12|[-2.3039491471256...|
| 13|[-5.2289678333324...|
| 14|[-1.6661411464084...|
| 18|[-1.2120289770257...|
| 38|[-2.5447459642824...|
| 67|[-1.1096207282233...|
| 70|[-1.4568195231945...|
| 93|[-0.8813424427756...|
|107|[-2.3664428998391...|
|148|[-2.7694230284943...|
|157|[-0.0987215878968...|
|161|[-1.4073323188082...|
|171|[-3.1400422558676...|
|186|[-1.9267669874678...|
|198|[-0.2626666579157...|
|203|[-0.9092396108795...|
|218|[-1.0866922469058...|
|221|[-3.2264572986080...|
|225|[-1.2842097114594...|
|232|[-1.1047228408632...|
+---+--------------------+
only showing top 20 rows

